### Tag Users from Candidate Support we calculated using clustering around the self-declared candidate supporters, and the sentiment the users expressed regarding candidate positions.

In [1]:
#Get classifications downloaded from s3
#!s3cmd sync s3://dse-dhofman/KMeans_results/ ./KMeans_results

In [2]:
!cat ./KMeans_results/tag_users_2016_3_10.csv | head

,user,supports,supports_tag
1,000000x6,none,Sanders
6,000dillon000,none,Clinton
11,000reginapeyto1,none,Trump
19,003a04f8c2054b7,none,Trump
27,007assassinator,none,Trump
31,007cigarjoe,Trump,Trump
38,007leclerc,none,Sanders
40,007llisav,none,Sanders
53,00_anabel,none,Trump
cat: stdout: Broken pipe


In [3]:
candidates = ['Sanders','Clinton','Trump','Rubio','Cruz']

In [4]:
prequery = """CREATE INDEX ON :User_{0}(screen_name)"""

In [5]:
partitions = ['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15',\
              '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', '2016_2_2', '2016_2_3', \
              '2016_2_4', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9', '2016_2_10',\
              '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', \
              '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26', '2016_2_27', \
              '2016_3_1', '2016_3_2','2016_3_3', '2016_3_4', '2016_3_5', '2016_3_6', '2016_3_7', \
              '2016_3_8', '2016_3_9', '2016_3_10', '2016_3_11']

print len(partitions)

40


In [6]:
# #run this ahead of time to create indexes
# from neo4jwriter import neo4j_writer
# writer = neo4j_writer('remote',logger)
# for partition in partitions:
#     prequery_formatted = prequery.format(partition)
#     writer.runcmd(prequery_formatted)

In [7]:
query="""MATCH (u:User_{0}) \
WHERE u.screen_name in {1} \
SET u.supports = '{2}' \
RETURN count(u)"""

In [8]:
from neo4jwriter import neo4j_writer
import logging
import time
import os
import sys


class supportsreader:

    def __init__(self,logger):
        self.supportsfolder()
        self.logger = logger
        
    def supportsfolder(self):
        self.supportsfolder = './KMeans_results'
    
    def supportsfiles(self):      
        return [(self.supportsfolder +'/'+afile) \
                for afile in os.listdir(self.supportsfolder) \
               if ('DS_Store' not in afile)]
#         return ['./KMeans_results/tag_users_2016_3_10.csv']
    
    def partitionfromfilename(self,filename):
        afile = filename.split('/')[2]
        print afile
        afilepartition = afile.lstrip('./KMeans_results/tag_users_').rstrip('.csv')
        print afilepartition
        return afilepartition
    
    def formatline(self,line):
        values = line.rstrip('\n').split(',')
        d_item = {'screen_name':str(values[1]), 'supports':str(values[3])}
        return d_item

    def getitems(self):
        for filename in self.supportsfiles():
            self.logger.info('reading file {0}'.format(filename))
            #sys.stdout.flush()
            
            partition = self.partitionfromfilename(filename)

            with open(filename, 'rb') as f:
                for line in f: 
                    #self.logger.info('read line in file {0}'.format(filename))
                    #sys.stdout.flush()
                    formatted_item = self.formatline(line)
                    formatted_item['partition']=partition
                    yield formatted_item

class databatch:
    
    def __init__(self,query,writer,logger):
        self.data = []
        self.query = query
        self.writer = writer
        self.requestcount = 0
        self.resultcount = 0
        self.logger = logger
        
    def add(self, item):
        self.data.append(item)
        
    def items(self):
        return self.data
    
    def clear(self):
        self.data = []

    def startcount(self):
        self.requestcount = 0
        self.resultcount = 0
    
    #sort by tweetid to optimize performance of updates
    def sorteddata(self):
        return sorted(self.data, key=lambda k: k['screen_name'])
    
    def supportsdata(self):
        #print self.data[0]
        supportsoutput = {}
        
        for candidate in candidates:
            supportsoutput[candidate] = [{'screen_name':item['screen_name'],'partition':item['partition']}\
                                         for item in self.data if (item['supports']==candidate)]
            
        return supportsoutput
    
        
    def processbatch(self):

        try:
            candidates = self.supportsdata()
            
            for k,v in candidates.iteritems():
                values = v
                supports = k
                
                for partition in list(set([value['partition'] for value in values])):
                    
                    screen_names = [value['screen_name'] for value in values if (value['partition']==partition)]

                    query_formatted = self.query.format(partition, screen_names, supports)

                    if (len(values)>0):
                        #print query_formatted, len(screen_names)
                        count = self.writer.runcmd_result(query_formatted)
                        self.requestcount = self.requestcount + len(values)
                        self.resultcount = self.resultcount + count
            
        except Exception as e:
            self.logger.info('Error writing batch {0}'.format(e))
            
        self.clear()


#set up logging
logger = logging.getLogger('TagUsersByCandidate_support-local')
hdlr = logging.FileHandler('./supports_local.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)


batchsize = 200
writer = neo4j_writer('remote',logger)
reader = supportsreader(logger)
    
start = time.time() 
data = databatch(query,writer,logger)

for d_item in reader.getitems():

    data.add(d_item)

    if (len(data.items()) >= batchsize):
        data.processbatch()
        logger.info('processing time after batch: {0}, request count: {1}, result count: {2}'.format(time.time()-start, \
        data.requestcount, data.resultcount))


if (len(data.items()) > 0):
    data.processbatch()
    logger.info('processing time after batch: {0}, request count: {1}, result count: {2}'.format(time.time()-start, \
        data.requestcount, data.resultcount))


    print time.time() - start,data.requestcount,data.resultcount




INFO:TagUsersByCandidate_support-local:reading file ./KMeans_results/tag_users_2015_10_11.csv
INFO:TagUsersByCandidate_support-local:processing time after batch: 1.27989792824, request count: 199, result count: 132
INFO:TagUsersByCandidate_support-local:processing time after batch: 1.91721582413, request count: 399, result count: 260
INFO:TagUsersByCandidate_support-local:processing time after batch: 2.58002591133, request count: 599, result count: 400
INFO:TagUsersByCandidate_support-local:processing time after batch: 3.23868203163, request count: 799, result count: 533
INFO:TagUsersByCandidate_support-local:processing time after batch: 3.83184981346, request count: 999, result count: 675
INFO:TagUsersByCandidate_support-local:processing time after batch: 4.45772194862, request count: 1199, result count: 815
INFO:TagUsersByCandidate_support-local:processing time after batch: 5.07708287239, request count: 1399, result count: 947
INFO:TagUsersByCandidate_support-local:processing time af

tag_users_2015_10_11.csv
2015_10_11
tag_users_2015_10_12.csv

INFO:TagUsersByCandidate_support-local:processing time after batch: 21.8189549446, request count: 7197, result count: 4876
INFO:TagUsersByCandidate_support-local:processing time after batch: 22.6137578487, request count: 7397, result count: 5071
INFO:TagUsersByCandidate_support-local:processing time after batch: 23.4901919365, request count: 7597, result count: 5264
INFO:TagUsersByCandidate_support-local:processing time after batch: 24.4078278542, request count: 7797, result count: 5461
INFO:TagUsersByCandidate_support-local:processing time after batch: 25.2002379894, request count: 7997, result count: 5658
INFO:TagUsersByCandidate_support-local:processing time after batch: 26.2016170025, request count: 8197, result count: 5853
INFO:TagUsersByCandidate_support-local:processing time after batch: 27.0945258141, request count: 8397, result count: 6051
INFO:TagUsersByCandidate_support-local:processing time after batch: 27.9289050102, request count: 8597, result count: 6249
INFO:TagUsersByC


2015_10_12
tag_users_2015_10_13.csv

INFO:TagUsersByCandidate_support-local:processing time after batch: 105.824658871, request count: 30595, result count: 27483
INFO:TagUsersByCandidate_support-local:processing time after batch: 106.852469921, request count: 30795, result count: 27652
INFO:TagUsersByCandidate_support-local:processing time after batch: 107.784054041, request count: 30995, result count: 27831
INFO:TagUsersByCandidate_support-local:processing time after batch: 108.751749992, request count: 31195, result count: 27995
INFO:TagUsersByCandidate_support-local:processing time after batch: 109.652052879, request count: 31395, result count: 28134
INFO:TagUsersByCandidate_support-local:processing time after batch: 110.603224039, request count: 31595, result count: 28261
INFO:TagUsersByCandidate_support-local:processing time after batch: 111.587990046, request count: 31795, result count: 28388
INFO:TagUsersByCandidate_support-local:processing time after batch: 112.368587017, request count: 31995, result count: 28535


...


2016_3_7
tag_users_2016_3_8.csv

INFO:TagUsersByCandidate_support-local:processing time after batch: 7284.25462604, request count: 2358928, result count: 2247590
INFO:TagUsersByCandidate_support-local:processing time after batch: 7285.35724592, request count: 2359128, result count: 2247787
INFO:TagUsersByCandidate_support-local:processing time after batch: 7286.26705289, request count: 2359328, result count: 2247982
INFO:TagUsersByCandidate_support-local:processing time after batch: 7287.17444491, request count: 2359528, result count: 2248177
INFO:TagUsersByCandidate_support-local:processing time after batch: 7288.22707391, request count: 2359728, result count: 2248369
INFO:TagUsersByCandidate_support-local:processing time after batch: 7289.27764893, request count: 2359928, result count: 2248562
INFO:TagUsersByCandidate_support-local:processing time after batch: 7290.30411386, request count: 2360128, result count: 2248748
INFO:TagUsersByCandidate_support-local:processing time after batch: 7291.41493988, request count:


2016_3_8
tag_users_2016_3_9.csv

INFO:TagUsersByCandidate_support-local:processing time after batch: 7525.12672591, request count: 2437326, result count: 2322727
INFO:TagUsersByCandidate_support-local:processing time after batch: 7525.86760783, request count: 2437526, result count: 2322923
INFO:TagUsersByCandidate_support-local:processing time after batch: 7526.58523297, request count: 2437726, result count: 2323121
INFO:TagUsersByCandidate_support-local:processing time after batch: 7527.38064098, request count: 2437926, result count: 2323321
INFO:TagUsersByCandidate_support-local:processing time after batch: 7528.21517897, request count: 2438126, result count: 2323520
INFO:TagUsersByCandidate_support-local:processing time after batch: 7528.99980092, request count: 2438326, result count: 2323718
INFO:TagUsersByCandidate_support-local:processing time after batch: 7529.82827187, request count: 2438526, result count: 2323916
INFO:TagUsersByCandidate_support-local:processing time after batch: 7530.57857299, request count:


2016_3_9
7747.23663092 2509905 2394692
